In [76]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import copy
import cv2
import keras
from string import printable
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, MaxPooling1D, UpSampling2D, Flatten, LSTM, SimpleRNN, Embedding, Lambda, GlobalMaxPooling1D
from keras.models import Model, Sequential
from keras.datasets import fashion_mnist
from keras import backend as K
from keras.layers.core import Dense, Dropout, Activation, Flatten, Reshape
from keras.optimizers import SGD, RMSprop
from keras.utils import np_utils
from keras.regularizers import l2
from keras.layers.convolutional import Conv1D, Conv2D, MaxPooling2D, ZeroPadding2D, AveragePooling2D
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from PIL import Image
from keras.applications import VGG16
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.optimizers import RMSprop
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from keras import callbacks
from keras.callbacks import TensorBoard
from keras.preprocessing.sequence import pad_sequences

In [55]:
# read data
benign_list = []
malicious_list = []
char_set = set()
with open("benign-urls.txt") as files:
    lines = files.readlines()
    for line in lines:
        if not line.startswith("#"):
            benign_list.append(line.strip())
            for c in line:
                char_set.add(c)
            max_len = max(max_len, len_urls)
with open("malicious-urls.txt") as files:
    lines = files.readlines()
    for line in lines:
        malicious_list.append(line.strip())
        for c in line:
            char_set.add(c)
        max_len = max(max_len, len_urls)

In [56]:
# combine and split data
chars = sorted(list(char_set))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))
df_b = pd.DataFrame(benign_list, columns=['url'])
df_m = pd.DataFrame(malicious_list, columns=['url'])
X = pd.concat([df_b, df_m])
y = np.concatenate([np.ones(len(benign_list)), np.zeros(len(malicious_list))])
y = np.array(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True)

In [49]:
#3.1

In [50]:
X_train_rnn = np.zeros((len(X_train), max_len, len(chars)), dtype=np.uint8)
X_test_rnn = np.zeros((len(X_test), max_len, len(chars)), dtype=np.uint8)
for i, seq in enumerate(list(X_train)):
    for t, char in enumerate(seq):
        X_train_rnn[i, t, char_indices[char]] = 1
for i, seq in enumerate(list(X_test)):
    for t, char in enumerate(seq):
        X_test_rnn[i, t, char_indices[char]] = 1

In [51]:
model_rnn = Sequential()
model_rnn.add(SimpleRNN(32, input_shape=(86, len(chars))))
model_rnn.add(Dense(1, activation="softmax"))
model_rnn.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['acc'])
model_rnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_3 (SimpleRNN)     (None, 32)                3232      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 33        
Total params: 3,265
Trainable params: 3,265
Non-trainable params: 0
_________________________________________________________________


In [52]:
model_rnn.fit(X_train_rnn, y_train, epochs=10, batch_size=128)

Epoch 1/10
47117/47117 [==============================] - 9s 197us/step - loss: 0.3038 - acc: 0.9809
Epoch 2/10
47117/47117 [==============================] - 8s 176us/step - loss: 0.3038 - acc: 0.9809
Epoch 3/10
47117/47117 [==============================] - 9s 189us/step - loss: 0.3038 - acc: 0.9809
Epoch 4/10
47117/47117 [==============================] - 9s 193us/step - loss: 0.3038 - acc: 0.9809
Epoch 5/10
47117/47117 [==============================] - 9s 186us/step - loss: 0.3038 - acc: 0.9809
Epoch 6/10
47117/47117 [==============================] - 9s 189us/step - loss: 0.3038 - acc: 0.9809
Epoch 7/10
47117/47117 [==============================] - 9s 198us/step - loss: 0.3038 - acc: 0.9809
Epoch 8/10
47117/47117 [==============================] - 9s 186us/step - loss: 0.3038 - acc: 0.9809
Epoch 9/10
47117/47117 [==============================] - 9s 185us/step - loss: 0.3038 - acc: 0.9809
Epoch 10/10
47117/47117 [==============================] - 9s 184us/step - loss: 0.3038 - a

In [53]:
print("Train classification accuracy rate %0.05f" % model_rnn.evaluate(X_train_rnn, y_train, verbose=0)[1])
print("Test classification accuracy rate %0.05f" % model_rnn.evaluate(X_test_rnn, y_test, verbose=0)[1])

Train classification accuracy rate 0.98094
Test classification accuracy rate 0.98029


In [ ]:
#3.2

In [81]:
X_train_cnn = np.zeros((len(X_train), max_len), dtype=np.uint8)
X_test_cnn = np.zeros((len(X_test), max_len), dtype=np.uint8)
for i, seq in enumerate(list(X_train)):
    for t, char in enumerate(seq):
        X_train_cnn[i, t] = 1
for i, seq in enumerate(list(X_test)):
    for t, char in enumerate(seq):
        X_test_cnn[i, t] = 1

In [85]:
embedding = Embedding(len(chars), 32, input_length=max_len, trainable=True)
inputs = Input(shape=(max_len,), dtype='int32')
x = embedding(inputs)
x = Conv1D(64, 3, activation='relu')(x)
x = MaxPooling1D(3)(x)
x = Conv1D(64, 3, activation='relu')(x)
x = GlobalMaxPooling1D()(x)
x = Dense(1, activation='sigmoid')(x)
model_cnn = Model(inputs, x)
model_cnn.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['acc'])
model_cnn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 86)                0         
_________________________________________________________________
embedding_15 (Embedding)     (None, 86, 32)            2176      
_________________________________________________________________
conv1d_18 (Conv1D)           (None, 84, 64)            6208      
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 28, 64)            0         
_________________________________________________________________
conv1d_19 (Conv1D)           (None, 26, 64)            12352     
_________________________________________________________________
global_max_pooling1d_4 (Glob (None, 64)                0         
_________________________________________________________________
dense_18 (Dense)             (None, 1)                 65        
Total para

In [86]:
model_cnn.fit(X_train_cnn, y_train, epochs=10, batch_size=128)

Epoch 1/10
47117/47117 [==============================] - 9s 197us/step - loss: 0.1103 - acc: 0.9810
Epoch 2/10
47117/47117 [==============================] - 8s 173us/step - loss: 0.0949 - acc: 0.9810
Epoch 3/10
47117/47117 [==============================] - 9s 194us/step - loss: 0.0949 - acc: 0.9810
Epoch 4/10
47117/47117 [==============================] - 8s 165us/step - loss: 0.0951 - acc: 0.9810
Epoch 5/10
47117/47117 [==============================] - 9s 194us/step - loss: 0.0949 - acc: 0.9810
Epoch 6/10
47117/47117 [==============================] - 9s 189us/step - loss: 0.0950 - acc: 0.9810
Epoch 7/10
47117/47117 [==============================] - 8s 176us/step - loss: 0.0949 - acc: 0.9810
Epoch 8/10
47117/47117 [==============================] - 9s 195us/step - loss: 0.0949 - acc: 0.9810
Epoch 9/10
47117/47117 [==============================] - 8s 171us/step - loss: 0.0949 - acc: 0.9810
Epoch 10/10
47117/47117 [==============================] - 9s 199us/step - loss: 0.0948 - a

In [87]:
print("Train classification accuracy rate %0.05f" % model_cnn.evaluate(X_train_cnn, y_train, verbose=0)[1])
print("Test classification accuracy rate %0.05f" % model_cnn.evaluate(X_test_cnn, y_test, verbose=0)[1])

Train classification accuracy rate 0.98098
Test classification accuracy rate 0.98019


In [ ]:
#3.3

In [ ]:
rnn_result = model_rnn.predict(X_test_rnn).ravel()
(rnn_fp, rnn_tp, rnn_threshold) = roc_curve(y_test, rnn_result)
rnn_auc = auc(rnn_fp, rnn_tp)

cnn_result = model_cnn.predict(X_test_cnn).ravel()
(cnn_fp, cnn_tp, cnn_threshold) = roc_curve(y_test, cnn_result)
cnn_auc = auc(cnn_fp, cnn_tp)
plt.figure()
plt.title('ROC')
plt.xlabel('FP')
plt.ylabel('TP')
plt.plot([0, 1], [0, 1], linestyle='dashed')
plt.plot(rnn_fp, rnn_tp, label='RNN ROC area = %0.4f' % rnn_auc)
plt.plot(cnn_fp, cnn_tp, label='CNN ROC area = %0.4f' % cnn_auc)
plt.legend(loc="best")
plt.show()